In [3]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random
import numpy as np

# Load the dataset
df = pd.read_csv("data/The_Cancer_data_1500_V2.csv")

# Drop the 'Diagnosis' column
df.pop("Diagnosis")

# Get the columns and a random row
columns = df.columns.values
rand = random.randint(0, len(df) - 1)

# Extract the features and create the inputs dictionary
features = df.iloc[rand].to_dict()

# Ensure all values are Python native types
for key in features:
    if isinstance(features[key], (np.integer, np.int64)):
        features[key] = int(features[key])
    elif isinstance(features[key], (np.floating, np.float64)):
        features[key] = float(features[key])

# Ensure specific fields are of the correct data type
features['Age'] = int(features['Age'])
features['Gender'] = int(features['Gender'])
features['Smoking'] = int(features['Smoking'])
features['GeneticRisk'] = int(features['GeneticRisk'])
features['CancerHistory'] = int(features['CancerHistory'])

def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, "utf-8")]
        ),
    )
    
def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )
    
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

def prepare_json(inputs: dict):
    feature_spec = {}
    
    for key, value in inputs.items():
        if isinstance(value, float):
            feature_spec[key] = float_feature(value)
        elif isinstance(value, int):
            feature_spec[key] = int_feature(value)
        elif isinstance(value, str):
            feature_spec[key] = string_feature(value)
        else:
            print(f"Unsupported data type for {key}: {type(value)}")
            
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
        
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

def make_predictions(inputs):
    json_data = prepare_json(inputs)
    
    endpoint = "https://male-kaleena-naba231-54e6cd2a.koyeb.app/v1/models/cancer-failure-model:predict"
    response = requests.post(endpoint, data=json_data)

    try:
        response_data = response.json()
        if "predictions" in response_data:
            prediction = response_data["predictions"][0][0]
            if prediction < 0.6:
                return "No Cancer"
            else:
                return "Cancer"
        else:
            print("Unexpected response format:")
            print(response_data)
            return "Prediction Error"
    except Exception as e:
        print(f"Error processing prediction response: {e}")
        return "Prediction Error"

# Example usage:
print("Example input: ",features)
print()
result = make_predictions(features)
print("Prediction result:", result)


Example input:  {'Age': 35, 'Gender': 1, 'BMI': 26.5134874318535, 'Smoking': 0, 'GeneticRisk': 2, 'PhysicalActivity': 4.582048052522159, 'AlcoholIntake': 1.1567435660007526, 'CancerHistory': 0}

Prediction result: Cancer
